### Final

In [ ]:
## Start
import pandas as pd
lpath = './'
sem_df = pd.read_csv(lpath + 'sem')
lex_df = pd.read_csv(lpath + 'lex')
beh_df = pd.read_csv(lpath + 'beh')

In [ ]:
marks_path = './data/ranking-long-tail-queries-fall-2020/train.marks.tsv/train.marks.tsv'
marks_df = pd.read_csv(marks_path, header=None, sep='\t')
marks_df = marks_df.rename(columns={0: 'QueryId', 
                         1: 'DocumentId',
                         2: 'mark'})

sample_path = './data/ranking-long-tail-queries-fall-2020/sample.csv/sample.csv'
sample_df = pd.read_csv(sample_path, sep=',')


In [ ]:
all_df = sem_df.merge(lex_df).merge(beh_df)

train_df = marks_df.merge(all_df, how='left')
test_df = sample_df.merge(all_df, how='left')

In [ ]:
curr_valnames = set(all_df.columns) - set(['DocumentId', 'QueryId', 'mark'])

In [ ]:
!pip install catboost

In [ ]:
###!
import numpy as np
from catboost import CatBoost, Pool
train_df_with_marks = train_df
test = test_df
#
X_train = train_df_with_marks[curr_valnames].values
y_train = train_df_with_marks["mark"].values
X_test = test[curr_valnames].values

full = Pool(
    data = train_df_with_marks[curr_valnames].values,
    label = train_df_with_marks.mark.values / 5,
    group_id = train_df_with_marks.QueryId.values
)
#
parameters = {
    'iterations': 1000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=5'],
    'verbose': False,
    'random_seed': 1313,
    'depth': 7,
    'bootstrap_type': 'Bernoulli',
    'loss_function': 'YetiRank',
    'subsample':0.925
}
#

model = CatBoost(parameters)
print('catboost fitting...')
model.fit(full, eval_set = [full])
print('catboost fit finished...')


##
import lightgbm as gbm
group_train = [query for query in np.bincount(train_all.QueryId.tolist()) if query > 0]


params={'boosting_type': 'gbdt',
        'score': 'ndcg@5',
        'n_estimators': 100,
        'subsample': 0.9,
        'learning_rate': 0.1,
        'num_leaves': 63}
Ranker1 = gbm.LGBMRanker(**params)
print('lgbm100 fitting...')
Ranker1.fit(X_train, y_train, group=group_train, verbose=1)
print('lgbm100 fit finished')

params={'boosting_type':'gbdt',
        'score':'ndcg@5',
        'n_estimators':3000,
        'subsample':0.95,
        'learning_rate':0.007,
        'num_leaves':29}
Ranker = gbm.LGBMRanker(**params)
print('lgbm3000 fitting...')
Ranker.fit(X_train, y_train, group=group_train, verbose=1)
print('lgbm3000 fit finished...')


In [ ]:
test_data = Pool(data = X_test)
cb_pred = model.predict(test_data)
lgbm3000_pred = Ranker.predict(X_test)
lgbm100_pred = Ranker1.predict(X_test)
sum_pred = cb_pred + lgbm3000_pred + lgbm100_pred

test["predicted_mark"] = sum_pred
result = test.sort_values(by=['QueryId', "predicted_mark"], ascending = [True, False])
result[['QueryId', 'DocumentId']].to_csv('final_subm', sep=',', index=None)